In [1]:
import pandas as pd
import os
import glob
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category = FutureWarning)
from tqdm import tqdm
import sys
import re

sys.path.insert(0, '../../../src/utils')
from import_data import *

def summ_followers2(df):
    cols = ['eng', 'n_eng']

    df_agg = df[['username'] + 
        cols].groupby(['username']).sum().reset_index()

    return df_agg

#This I do recommend running it first for one country, restart kernel, and run for the other country:

In [2]:
country = 'SA'

base1 = pd.read_parquet(f'../../../data/04-analysis/{country}/treatment_info/information_batch1.parquet')

df_final = get_baseline_data_b1(country, type_data = 'predicted', base_path = '../../../')
df_final['eng'] = np.where((df_final['lang'] == 'en'), 1, 0)
df_final['n_eng'] = np.where((df_final['lang'] != 'en'), 1, 0)
df_final.rename(columns = {'handle': 'username', 
                       'author_id':'follower_id'}, inplace = True)
df_RT = df_final[(df_final['text'].str.contains('RT @', case=True, regex=False)) & 
                 (df_final['total_comments'] == 0)].reset_index(drop=True)

df_no_rt = df_final[(~df_final['text'].str.contains('RT @', 
                    case=True, regex=False)) | 
                    (df_final['total_comments'] > 0)].reset_index(drop=True)

df_final_agg = summ_followers2(df_final).reset_index(drop = True)
df_rt_agg = summ_followers2(df_RT).reset_index(drop=True)
df_no_rt_agg = summ_followers2(df_no_rt).reset_index(drop=True)

cols = ['eng', 'n_eng']

df_final_agg.rename(columns = 
        {col: col + '_base' for col in df_rt_agg.columns if col in cols}, 
        inplace=True)

df_rt_agg.rename(columns = 
        {col: col + '_rt_base' for col in df_rt_agg.columns if col in cols}, 
        inplace=True)

df_no_rt_agg.rename(columns = 
        {col: col + '_no_rt_base' for col in df_no_rt_agg.columns if col in cols}, 
        inplace=True)

base1 = base1.merge(df_final_agg, on=['username'], 
                    how='left')

base1 = base1.merge(df_rt_agg, on=['username'], 
                    how='left')

base1 = base1.merge(df_no_rt_agg, on=['username'], 
                    how='left')

cols_base = [col for col in base1.columns if '_base' in col]
for x in cols_base:
    base1[x] = np.where(base1[x].isnull(), 0, base1[x])
    
base1.to_parquet(f'../../../data/04-analysis/{country}/baseline/baseline_english_batch1.parquet')

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [01:58<00:00,  1.60s/it]


In [2]:
country = 'KE'

base1 = pd.read_parquet(f'../../../data/04-analysis/{country}/treatment_info/information_batch1.parquet')

df_final = get_baseline_data_b1(country, type_data = 'predicted', base_path = '../../../')
df_final['eng'] = np.where((df_final['lang'] == 'en'), 1, 0)
df_final['n_eng'] = np.where((df_final['lang'] != 'en'), 1, 0)
df_final.rename(columns = {'handle': 'username', 
                       'author_id':'follower_id'}, inplace = True)

df_RT = df_final[(df_final['text'].str.contains('RT @', case=True, regex=False)) & 
                 (df_final['total_comments'] == 0)].reset_index(drop=True)

df_no_rt = df_final[(~df_final['text'].str.contains('RT @', 
                    case=True, regex=False)) | 
                    (df_final['total_comments'] > 0)].reset_index(drop=True)

df_final_agg = summ_followers2(df_final).reset_index(drop = True)
df_rt_agg = summ_followers2(df_RT).reset_index(drop=True)
df_no_rt_agg = summ_followers2(df_no_rt).reset_index(drop=True)

cols = ['eng', 'n_eng']

df_final_agg.rename(columns = 
        {col: col + '_base' for col in df_rt_agg.columns if col in cols}, 
        inplace=True)

df_rt_agg.rename(columns = 
        {col: col + '_rt_base' for col in df_rt_agg.columns if col in cols}, 
        inplace=True)

df_no_rt_agg.rename(columns = 
        {col: col + '_no_rt_base' for col in df_no_rt_agg.columns if col in cols}, 
        inplace=True)

base1 = base1.merge(df_final_agg, on=['username'], 
                    how='left')

base1 = base1.merge(df_rt_agg, on=['username'], 
                    how='left')

base1 = base1.merge(df_no_rt_agg, on=['username'], 
                    how='left')

cols_base = [col for col in base1.columns if '_base' in col]
for x in cols_base:
    base1[x] = np.where(base1[x].isnull(), 0, base1[x])
    
base1.to_parquet(f'../../../data/04-analysis/{country}/baseline/baseline_english_batch1.parquet')

100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [04:00<00:00,  2.87s/it]
